# REST API Source Connections

One of the major new advances of SysML v2 is the definition of a standard API for server-side and headless clients to expose in order to support a standard way of loading model data in multiple applications. This notebook walks through PyMBE's approach to loading data from such an API.

In [ ]:
printing_level = "TRACE"
import requests
import re

## Public Instance example

This example is built around the public instance for the API, maintained at standards bodies and groups like OpenMBEE.

In [ ]:
host_url = "http://sysml2.intercax.com"
host_port = 9000

### Finding existing projects and commits

The top level of navigation for data in an API repository is the Project with multiple commits, as described by the platform independent model, pictured below.

![API PIM](../images/api_pim.png "API Platform Independent Model")

In [ ]:
projects_url = f"{host_url}:{host_port}/projects"
projects_url

In [ ]:
projects_response = requests.get(projects_url)

In [ ]:
desired_project = "3a-Function-based Behavior"

In [ ]:
projects_kvp = {}
if printing_level == "TRACE":
    print(f"Downloading data from {host_url} ...")
if projects_response.status_code == 200:
    if printing_level == "TRACE":
        print(f"Download returned with OK status code.")
    projects_kvp = projects_response.json()
    filtered_project = [project for project in projects_kvp
                        if project['name'].startswith(desired_project)]
    if printing_level == "TRACE":
        print(f"Filtered projects include: {[project['name'] for project in filtered_project]}")

In [ ]:
commits_url = f"{projects_url}/{filtered_project[0]['@id']}/commits"
commits_url

In [ ]:
commits_response = requests.get(commits_url)
if commits_response.status_code == 200:
    if printing_level == "TRACE":
        print(f"Download returned with OK status code.")
    commits_kvp = commits_response.json()
    if printing_level == "TRACE":
        print(f"Found {len(commits_kvp)} commits under this project.")

### Acquiring Elements

Under a given commit, multiple elements can be found. There may be a large number of elements in the model, so moving through multiple pages of elements may be needed.

In [ ]:
elements_url = f"{commits_url}/{commits_kvp[0]['@id']}/elements"
elements_url

The reference to the next page is contained in response headers. We can parse it with a regular expression.

In [ ]:
_next_url_regex = re.compile(r'<(http://.*)>; rel="next"')

In [ ]:
url = elements_url
collected_elements = []

while url:
    elements_response = requests.get(url)
    if elements_response.status_code == 200:
        link = elements_response.headers.get("Link")
        if not link:
            if printing_level == "TRACE":
                print("No more pages found.")
            break
        urls = _next_url_regex.findall(link)
        if printing_level == "TRACE":
            print(f"Download returned with OK status code.")
            print(f"Link to next page found at: {urls}")
        if len(urls) == 1:
            url = urls[0]
        else:
            url = None
        if printing_level == "TRACE":
            print("Adding found elements to collected result.")
        
        collected_elements += elements_response.json()
        
if printing_level == "TRACE":
    print(f"Downloaded {len(collected_elements)} elements.")

### Local Save

The collected dictionary of elements can be rendered back into JSON and saved to the local filesystem.

In [ ]:
import json

In [ ]:
with open('run_examples_data/downloaded_elements.json', 'w') as fp:
    json.dump(collected_elements, fp)